This notebook contains efficient implementations of the Collaborative Filtering techniques. This implementation makes use of sparse representations of the matrices. The algorithms take very long to implement on the larger version of the dataset. In this notebook, we demonstrate the use of these algos on the smaller version of the dataset. The code is also available as .py files which can be used on the larger version of the dataset.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
# filename = '../../data/ml-20m/ratings.csv'
filename = '../../data/ml-latest-small/ratings.csv'
df = pd.read_csv(filename)

n_users = df['userId'].unique().shape[0]
n_items = df['movieId'].unique().shape[0]
print("Number of unique users: %d" % n_users)
print("Number of unique movies: %d" % n_items)

Number of unique users: 671
Number of unique movies: 9066


In [3]:
import scipy.sparse as sparse

# Split the data into two parts, train and test. Randomly sample 10% of the entries in the csv file as test data and 
# the rest of the data will be used as training data

# Also create a validation partition for tuning the hyperparameters
df_train = df.sample(frac=0.9)
df_test = df.loc[~df.index.isin(df_train.index)]
print(df_train.shape)
print(df_test.shape)

(90004, 4)
(10000, 4)


In [4]:
# README file for the dataset: http://files.grouplens.org/datasets/movielens/ml-20m-README.html
# User-ids are in the range (1, 138493). We just subract 1 from each userId to convert the range to (0,138492)
# Total number of movies are 27278 but the the range of movieIds is bigger than (1,27278)
# We need to map the movieIds to the range (0,27277)
# Only movies with at least one rating or tag are included in the dataset. As we see above, the number of unique movies
# for which we have atleast one rating is 26744 

print(df['movieId'].unique().shape)
ind = 0
movie_list = [] # List which is reverse of movie_dict, contains original movieId at index 'new id'
movie_dict = {}   # Dictionary from original movieId to new id
for movieId in df['movieId'].unique():
    movie_list.append(movieId)
    movie_dict[movieId] = ind
    ind += 1

print(len(movie_list))        

(9066,)
9066


In [5]:
# Create sparse matrix for the training data
print(len(df_train['movieId'].tolist()))

movie_id_list = list(map(lambda x: movie_dict[x], df_train['movieId'].tolist()))
print(len(movie_id_list))
user_id_list = list(map(lambda x: x - 1, df_train['userId'].tolist()))
print(len(user_id_list))

90004
90004
90004


In [6]:
# Create sparse matrix for the test data
print(len(df_test['movieId'].tolist()))

test_movie_id_list = list(map(lambda x: movie_dict[x], df_test['movieId'].tolist()))
#print(len(test_movie_id_list))
test_user_id_list = list(map(lambda x: x - 1, df_test['userId'].tolist()))
#print(len(test_user_id_list))

sparse_test = sparse.coo_matrix((df_test['rating'].tolist(), (test_user_id_list,test_movie_id_list)),shape=(n_users,n_items))
print(sparse_test.nnz)

10000
10000


In [7]:
sparse_train = sparse.coo_matrix((df_train['rating'].tolist(),(user_id_list, movie_id_list)),shape=(n_users,n_items))
print(sparse_train.shape)
print(sparse_train.nnz)

(671, 9066)
90004


In [8]:
user_sum = sparse_train.sum(axis=1)
print(user_sum.shape)

(671, 1)


In [9]:
sparse_train_csr = sparse_train.tocsr()
data = sparse_train_csr.data
indices = sparse_train_csr.indices
indptr = sparse_train_csr.indptr
print(type(data))
print(type(indices))
print(type(indptr))
print(indices.shape)
print(indptr.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(90004,)
(672,)


In [10]:
data_useravg = np.empty(shape=data.shape,dtype=np.float64)
print(data_useravg.shape)

# Compute the average rating for each user
for user_num in range(indptr.shape[0] - 1):
    data_useravg[indptr[user_num]: indptr[user_num + 1]] = user_sum[user_num,0] / (indptr[user_num + 1] - indptr[user_num])
    
print(data_useravg[:25]) 

data_centered = data - data_useravg
print(data[:10] , data_useravg[:10], data_centered[:10])

(90004,)
[ 2.5         2.5         2.5         2.5         2.5         2.5         2.5
  2.5         2.5         2.5         2.5         2.5         2.5         2.5
  2.5         2.5         3.51470588  3.51470588  3.51470588  3.51470588
  3.51470588  3.51470588  3.51470588  3.51470588  3.51470588]
[ 2.5  3.   3.   2.   4.   2.   2.   3.5  2.   2.5] [ 2.5  2.5  2.5  2.5  2.5  2.5  2.5  2.5  2.5  2.5] [ 0.   0.5  0.5 -0.5  1.5 -0.5 -0.5  1.  -0.5  0. ]


In [11]:
# Create sparse CSR matrix of centered user data
sparse_train_ucentered = sparse.csr_matrix((data_centered,indices,indptr),shape=(n_users,n_items))

print(sparse_train_ucentered.nnz)
print(sparse_train_ucentered.shape)

90004
(671, 9066)


In [12]:
def latent_factor(sparse_ratings_coo, k=500, lambda1=1, lambda2=1, epochs=50, lr=0.1):
    
    # Initialize the latent factor matrices randomly
    Q = np.random.uniform(0,0.1,size=(sparse_ratings_coo.shape[0],k))
    P = np.random.uniform(0,0.1,size=(sparse_ratings_coo.shape[1],k))
    
    for ep in range(1,epochs+1):
        print("Epoch " + str(ep))
        for i,j,v in zip(sparse_ratings_coo.row, sparse_ratings_coo.col, sparse_ratings_coo.data):
            err = v - Q[i,:].dot(P[j,:])
            
            grad_Q = 2 * (lambda2 * Q[i,:] - (err)*P[j,:])  
            grad_P = 2 * (lambda1 * P[j,:] - (err)*Q[i,:])
            
            Q[i,:] -= lr * grad_Q
            P[j,:] -= lr * grad_P                                           
                                                       
    return Q,P    

In [13]:
def latent_factor_predict(sparse_test_coo, Q, P):
    """ Get the ratings for the test data points using the learnt Q and P matrices
    """
    pred = []
    
    for i,j in zip(sparse_test_coo.row, sparse_test_coo.col):
        pred.append(Q[i,:].dot(P[j,:]))
    
    return pred

In [15]:
Q,P = latent_factor(sparse_train,epochs=20)

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20


In [16]:
pred = latent_factor_predict(sparse_test, Q, P)
actual = sparse_test.data
mse = mean_squared_error(pred,actual)
print(mse)

2.45798708389


In [26]:
# Generate a toy sparse matrix for trying out methods

trial_mat_coo = sparse.random(10,7,density=0.25)
print(trial_mat_coo.nnz)
print(trial_mat_coo)


17
  (1, 0)	0.192160153478
  (1, 4)	0.931613711497
  (8, 2)	0.899914321873
  (6, 4)	0.927588282635
  (2, 1)	0.544007712114
  (4, 4)	0.2008087566
  (4, 2)	0.310863129116
  (4, 3)	0.610998632784
  (0, 0)	0.336660192172
  (9, 1)	0.160311436188
  (2, 5)	0.0465962755326
  (4, 0)	0.281276183295
  (3, 2)	0.331769992642
  (2, 3)	0.462017330272
  (6, 6)	0.177778118874
  (2, 4)	0.400253235155
  (6, 1)	0.945580065076


In [60]:
trial_mat_csr = trial_mat_coo.tocsr()
sample_row = trial_mat_csr.getrow(1).toarray()
print(sample_row.shape)

(1, 7)


In [39]:
similarity = sample_row.dot(trial_mat_csr.transpose())
print(similarity)
print(similarity.shape)
print(type(similarity))

  (0, 6)	0.864153962726
  (0, 2)	0.372881401941
  (0, 4)	0.241126265589
  (0, 1)	0.904829632034
  (0, 0)	0.0646926741975
(1, 10)
<class 'scipy.sparse.csr.csr_matrix'>


In [34]:
similarity.sort_indices() # Sorts indices, not the values
print(similarity)

  (0, 0)	0.0646926741975
  (0, 1)	0.904829632034
  (0, 2)	0.372881401941
  (0, 4)	0.241126265589
  (0, 6)	0.864153962726


In [35]:
sim_dense = similarity.todense()
print(type(sim_dense))
print(sim_dense.shape)
print(sim_dense)

<class 'numpy.matrixlib.defmatrix.matrix'>
(1, 10)
[[ 0.06469267  0.90482963  0.3728814   0.          0.24112627  0.
   0.86415396  0.          0.          0.        ]]


In [44]:
#norms = np.random.uniform(0,1,size=sim_dense.shape)
norms = np.ones(similarity.shape)
print(similarity)
similarity /= (norms + 1e-9)
print(similarity)
print(similarity.shape)
print(type(similarity))
sim_np = np.array(similarity).reshape(-1)
print(sim_np)
print(sim_np.shape)
sim_np.arg

[[ 0.06469267  0.90482963  0.3728814   0.          0.24112626  0.
   0.86415396  0.          0.          0.        ]]
[[ 0.06469267  0.90482963  0.3728814   0.          0.24112626  0.
   0.86415396  0.          0.          0.        ]]
(1, 10)
<class 'numpy.matrixlib.defmatrix.matrix'>
[ 0.06469267  0.90482963  0.3728814   0.          0.24112626  0.
  0.86415396  0.          0.          0.        ]
(10,)


In [45]:
def compute_norms(sparse_train_csr):
    """ Compute the norm of the rating vector for each user.
        These norm values will be used for normalizing during computation of cosine similarity
    """
    n_users = sparse_train_csr.shape[0]
    norms = np.empty(shape=(n_users,),dtype=np.float64)
    
    for i in range(n_users):
        row = sparse_train_csr.getrow(i)
        #getrow returns a 1 x n vector. Dot product gives a 1 x 1 numpy array and then we index using [0,0] to get a scalar
        norms[i] = np.sqrt(row.dot(row.T))[0,0]
    return norms    

In [50]:
def user_sim(userid, sparse_train_csr, norms):
    """ Compute the similarity between a particular user and all other users
        return an array/list containing the similarity with all other users
    """
    row = sparse_train_csr.getrow(userid)
    sim = row.dot(sparse_train_csr.transpose())
    
    sim /= (norms[userid] + 1e-9)
    sim /= (norms + 1e-9)
    
    #Convert from np.matrix datatype to a numpy ndarray, reshape to a vector and then return
    return np.array(sim).reshape(-1)    


In [69]:
def user_cf(userid, user_avg, user_sim, sparse_train_csr, k=100):
    """ user_sim is a numpy array of shape (n_users,)
    """
    
    top_k_users = user_sim.argsort()[-k-1:-1]
    top_k_sim = user_sim[top_k_users]
    
    # Initialize all predictions to the average rating given by that user
    pred = np.ones(sparse_train_csr.shape[1])
    pred *= user_avg
    
    #print(pred)
    
    top_k_ratings = np.empty((k,sparse_train_csr.shape[1]),dtype=np.float64)
    for i in range(k):
        top_k_ratings[i] = sparse_train_csr.getrow(top_k_users[i]).toarray()
        
    #print(top_k_ratings)    
        
    # Map of non-zero rating entries
    ratings_map = (top_k_ratings !=0).astype(int)
    #print(ratings_map)
    
    normalizer = top_k_sim.reshape((1,-1)).dot(ratings_map)
    #print(normalizer.shape)
    
    # pred is of shape (n_items,) and the other operand is of shape (1,n_items), hence the indexing [0,:]
    pred += (top_k_sim.reshape((1,-1)).dot(top_k_ratings) / (normalizer + 1e-9))[0,:]
    return pred

In [61]:
# Test above three methods on a toy example
trial_mat_coo = sparse.random(10,7,density=0.25)
trial_mat_csr = trial_mat_coo.tocsr()

# trial_norms = compute_norms(trial_mat_csr)
# print(trial_norms.shape)
# print(trial_norms)

# trial_usim = user_sim(1, trial_mat_csr, trial_norms)
# print(trial_usim.shape)
# print(trial_usim)

# trial_pred=user_cf(1, 5.0, trial_usim, trial_mat_csr, k=5)
# print(trial_pred.shape)
# print(trial_pred)

trial_test_coo = sparse.random(10,7, density=0.15)
trial_test_csr = trial_test_coo.tocsr()

In [72]:
def compute_usercf_MSE(sparse_train_csr, sparse_test_csr, k=100):    
    """ Compute MSE on the held out test data
    """
    # Compute the average rating for each user
    data_useravg = np.empty(shape=data.shape,dtype=np.float64)
    for user_num in range(indptr.shape[0] - 1):
        data_useravg[indptr[user_num]: indptr[user_num + 1]] = user_sum[user_num,0] / (indptr[user_num + 1] - indptr[user_num])
    
    norms = compute_norms(sparse_train_csr)    
    n_users = sparse_train_csr.shape[0]
    
    total_mse = 0
    
    for user in range(n_users):
        usim = user_sim(user, sparse_train_csr, norms)
        pred = user_cf(user, data_useravg[user], usim, sparse_train_csr,k)
        
        actual = sparse_test_csr.getrow(user).toarray().squeeze(axis=0)
        test_mask = (actual != 0).astype(int)
        
        pred = pred * test_mask
        mse_user = mean_squared_error(pred,actual)
        
        total_mse += (mse_user * np.count_nonzero(actual))
        
    return total_mse/sparse_test_csr.nnz             

In [71]:
trial_mse = compute_usercf_MSE(trial_mat_csr, trial_test_csr)
print(trial_mse)

2.74132967815


In [ ]:
sparse_test_csr = sparse_test.tocsr()
ucf_mse = compute_usercf_MSE(sparse_train_ucentered, sparse_test_csr, 100)
print(ucf_mse)